## Configuration

In [1]:
%pip install duckdb leafmap

Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
import leafmap

In [3]:
url = "https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip
To: /home/jovyan/work/BSABD/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 9.86MB/s]


Extracting files...


'/home/jovyan/work/BSABD/nyc_data.db.zip'

In [4]:
con = duckdb.connect("nyc_data.db")
con.install_extension("spatial")
con.load_extension("spatial")

In [5]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

## Questions

1. **What is the area of the 'West Village' neighborhood?** (Hint: The area is given in square meters. To get an area in hectares, divide by 10000. To get an area in acres, divide by 4047.)

In [8]:
con.sql("""
SELECT name, ST_Area(geom)
FROM nyc_neighborhoods
WHERE name ='West Village';
""")

┌──────────────┬────────────────────┐
│     NAME     │   st_area(geom)    │
│   varchar    │       double       │
├──────────────┼────────────────────┤
│ West Village │ 1044614.5296485958 │
└──────────────┴────────────────────┘

2. **What is the geometry type of ‘Pelham St’? The length?**

In [12]:
con.sql("""
    SELECT name, ST_GeometryType(geom), ST_Length(geom)
    FROM nyc_streets
    WHERE name = 'Pelham St';
""")

┌───────────┬───────────────────────┬───────────────────┐
│   NAME    │ st_geometrytype(geom) │  st_length(geom)  │
│  varchar  │     geometry_type     │      double       │
├───────────┼───────────────────────┼───────────────────┤
│ Pelham St │ MULTILINESTRING       │ 50.32314951660229 │
└───────────┴───────────────────────┴───────────────────┘

3. **What is the GeoJSON representation of the 'Broad St' subway station?**

In [13]:
con.sql("""
  SELECT ST_AsGeoJSON(geom)
  FROM nyc_subway_stations
  WHERE name = 'Broad St';
""")

┌──────────────────────────────────────────────────────────────────────┐
│                          st_asgeojson(geom)                          │
│                                 json                                 │
├──────────────────────────────────────────────────────────────────────┤
│ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │
└──────────────────────────────────────────────────────────────────────┘

4. **What is the total length of streets (in kilometers) in New York City?** (Hint: The units of measurement of the spatial data are meters, there are 1000 meters in a kilometer.)

con.sql("""
SELECT SUM(ST_Length(geom))/1000
FROM nyc_streets
""")

5. **What is the area of Manhattan in acres?** (Hint: both nyc_census_blocks and nyc_neighborhoods have a boroname in them.)

In [17]:
con.sql("""
SELECT SUM(ST_Area(geom))/4047
FROM nyc_census_blocks
WHERE BORONAME = 'Manhattan'
""")

┌─────────────────────────────┐
│ (sum(st_area(geom)) / 4047) │
│           double            │
├─────────────────────────────┤
│          14601.398721554795 │
└─────────────────────────────┘

* 6. **What is the most westerly subway station?**

In [18]:
con.sql("""
SELECT name, ST_X(geom)
FROM nyc_subway_stations
ORDER BY ST_X(geom)
LIMIT 1;
""")

┌─────────────┬───────────────────┐
│    NAME     │    st_x(geom)     │
│   varchar   │      double       │
├─────────────┼───────────────────┤
│ Tottenville │ 563292.1172580556 │
└─────────────┴───────────────────┘

7. **How long is 'Columbus Cir' (aka Columbus Circle)?**

In [20]:
con.sql("""
SELECT ST_Length(geom)
FROM nyc_streets
WHERE name = 'Columbus Cir';
""")

┌───────────────────┐
│  st_length(geom)  │
│      double       │
├───────────────────┤
│ 308.3419936909855 │
└───────────────────┘

8. **What is the length of streets in New York City, summarized by type?**

In [22]:
con.sql("""
SELECT type, SUM(ST_Length(geom)) AS length
FROM nyc_streets
GROUP BY type
ORDER BY length DESC;
""")

┌──────────────────────────────────────────────────┬────────────────────┐
│                       TYPE                       │       length       │
│                     varchar                      │       double       │
├──────────────────────────────────────────────────┼────────────────────┤
│ residential                                      │  8629870.337866059 │
│ motorway                                         │  403622.4781263628 │
│ tertiary                                         │  360394.8790513027 │
│ motorway_link                                    │   294261.419479668 │
│ secondary                                        │  276264.3038979258 │
│ unclassified                                     │  166936.3716044583 │
│ primary                                          │  135034.2330179469 │
│ footway                                          │  71798.48783780965 │
│ service                                          │ 28337.635038596007 │
│ trunk                               